# Machine Learning with Pipelines
### (Incomplete)
### Train, tune, and pickle models using Sweta's cleaned dataset
**Each pipeline will:**
- Standardize data with MinMax (vs standardscaler to accomodate binary dummy variables) 
- Select features with RFECV (recursive feature elimination with cross-validation)
- Tune hyperparameters with GridSearchCV

**Additional:**
- Evaluate performance
- Pickle
- Ensemble for flask predictor tool

In [78]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression, HuberRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

#### Pickle Code

In [82]:
# # save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))
 
# # some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

### Load Dummified Data

In [83]:
train = pd.read_csv('dummi data/train_final.csv')
test = pd.read_csv('dummi data/test_final.csv')
# test[test.isna().any(axis=1)]
test = test.dropna(axis=0)

In [84]:
X_train = train.drop(columns=['SalePrice', 'PID', 'Unnamed: 0'])
y_train = train['SalePrice']
X_test = test.drop(columns=['SalePrice', 'PID', 'Unnamed: 0'])
y_test = test['SalePrice']

### Feature Selection

### OLS Pipeline
- HuberRegressor to mitigate impact of outliers
- L2 Regularization

In [74]:
# hb = HuberRegressor(alpha=1)

# pipe = make_pipeline(
#     # standardize scale
# #     MinMaxScaler(), 
#     HuberRegressor()    
#     # feature selection 
# #     RFECV(hb, cv=4, scoring='accuracy')
# )

# # hyperparameter tuning
# grid_pipe = GridSearchCV(pipe,
#         param_grid={
#             'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#         }, cv=4, refit=True
#     )

# grid_pipe.fit(X_train, y_train)

In [75]:
# grid_pipe.best_score_

In [87]:
# hb = HuberRegressor()
# hb.fit(X_train,y_train)

# lm = LinearRegression()
# lm.fit(X_train, y_train)

In [88]:
pipe = make_pipeline(
    MinMaxScaler(),
    LinearRegression()
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('linearregression', LinearRegression())])

In [89]:
# evaluate model
y_pred = pipe.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

mean abs err: 15231.121806359592
r2: 0.8926889389619903


In [90]:
pickle.dump(pipe, open('Linear.sav', 'wb'))

### Bayesian Ridge Pipeline

In [88]:
pipe = make_pipeline(
    MinMaxScaler(),
    LinearRegression()
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('linearregression', LinearRegression())])

In [89]:
# evaluate model
y_pred = pipe.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

mean abs err: 15231.121806359592
r2: 0.8926889389619903


In [90]:
pickle.dump(pipe, open('Linear.sav', 'wb'))

### Random Forest Pipeline

In [ ]:


pickle.dump(model, open(filename, 'wb'))

### Gradient Booster Pipeline

In [ ]:


pickle.dump(model, open(filename, 'wb'))

### SVM SVC Pipeline

In [ ]:

pickle.dump(model, open(filename, 'wb'))

### PCA Pipeline

### Clustering Pipeline